In [ ]:
# Prediction Scores with Poisson Regression


Notebook attempting to predict the scores of any fixture given the teams that are playing it based on their performance in the previous season. Such an approach using Poisson Regression employs only historic data and ignores other factors. Nevertheless, it is a good estimator of a team's attacking and defensive strength.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as scipy
import random


### Load the data

In [3]:
df = pd.read_csv("./Data/Spain/SP1_13.csv")
df_14 = pd.read_csv("./Data/Spain/SP1_14.csv")

In [4]:
df.columns

Index([u'Div', u'Date', u'HomeTeam', u'AwayTeam', u'FTHG', u'FTAG', u'FTR',
       u'HTHG', u'HTAG', u'HTR', u'HS', u'AS', u'HST', u'AST', u'HF', u'AF',
       u'HC', u'AC', u'HY', u'AY', u'HR', u'AR', u'B365H', u'B365D', u'B365A',
       u'BWH', u'BWD', u'BWA', u'IWH', u'IWD', u'IWA', u'LBH', u'LBD', u'LBA',
       u'PSH', u'PSD', u'PSA', u'WHH', u'WHD', u'WHA', u'SJH', u'SJD', u'SJA',
       u'VCH', u'VCD', u'VCA', u'Bb1X2', u'BbMxH', u'BbAvH', u'BbMxD',
       u'BbAvD', u'BbMxA', u'BbAvA', u'BbOU', u'BbMx>2.5', u'BbAv>2.5',
       u'BbMx<2.5', u'BbAv<2.5', u'BbAH', u'BbAHh', u'BbMxAHH', u'BbAvAHH',
       u'BbMxAHA', u'BbAvAHA', u'PSCH', u'PSCD', u'PSCA'],
      dtype='object')

### Cleaning

We do not need information about division, data, referee and the betting odds from various companies for this method. 

In [5]:
res_13 = df.ix[:,:23]
res_13 = res_13.drop(['Div','Date'],axis=1)
res_14 = df_14.ix[:,:23]
res_14 = res_14.drop(['Div','Date'],axis=1)
bet_13 = df.ix[:,23:]

In [6]:
res_13.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,...,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H
0,Sociedad,Getafe,2,0,H,1,0,H,16,15,...,2,13,6,6,5,1,1,0,0,1.73
1,Valencia,Malaga,1,0,H,0,0,D,9,11,...,2,15,23,9,6,3,5,0,0,1.53
2,Valladolid,Ath Bilbao,1,2,A,1,1,D,8,13,...,3,10,8,5,5,1,0,0,0,2.50
3,Barcelona,Levante,7,0,H,6,0,H,22,4,...,1,15,16,9,3,1,3,0,0,1.08
4,Osasuna,Granada,1,2,A,0,2,A,14,13,...,4,15,17,7,6,1,4,0,0,2.00


### Dataframe to store the final league standings in 2013-14

We create a table with the goals scored, conceded, attacking strength, defensive strength of both teams.

Number of matches played at home = 19

Attacking strength at home (HAS) = (Goals scored at home / 19) / Average Number of goals at home in the season 

Defensive strength at home (HAS) = (Goals conceded at home / 19) / Average Number of goals conceded at home in the season 

In [7]:
#Team, Home Goals Score, Away Goals Score, Attack Strength, Home Goals Conceded, Away Goals Conceded, Defensive Strength
table_13 = pd.DataFrame(columns=('Team','HGS','AGS','HAS','AAS','HGC','AGC','HDS','ADS'))

In [8]:
avg_home_scored_13 = res_13.FTHG.sum() / 380.0
avg_away_scored_13 = res_13.FTAG.sum() / 380.0
avg_home_conceded_13 = avg_away_scored_13
avg_away_conceded_13 = avg_home_scored_13
print "Average number of goals at home",avg_home_scored_13
print "Average number of goals away", avg_away_scored_13
print "Average number of goals conceded at home",avg_away_conceded_13
print "Average number of goals conceded away",avg_home_conceded_13


Average number of goals at home 1.63157894737
Average number of goals away 1.11842105263
Average number of goals conceded at home 1.63157894737
Average number of goals conceded away 1.11842105263


In [9]:
res_home = res_13.groupby('HomeTeam')
res_away = res_13.groupby('AwayTeam')

In [10]:
table_13.Team = res_home.HomeTeam.all().values
table_13.HGS = res_home.FTHG.sum().values
table_13.HGC = res_home.FTAG.sum().values
table_13.AGS = res_away.FTAG.sum().values
table_13.AGC = res_away.FTHG.sum().values
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Almeria,26,17,NaN,NaN,31,40,NaN,NaN
1,Ath Bilbao,42,24,NaN,NaN,18,21,NaN,NaN
2,Ath Madrid,49,28,NaN,NaN,10,16,NaN,NaN
3,Barcelona,64,36,NaN,NaN,15,18,NaN,NaN
4,Betis,19,17,NaN,NaN,31,47,NaN,NaN


In [11]:
table_13.HAS = (table_13.HGS / 19.0) / avg_home_scored_13
table_13.AAS = (table_13.AGS / 19.0) / avg_away_scored_13
table_13.HDS = (table_13.HGC / 19.0) / avg_home_conceded_13
table_13.ADS = (table_13.AGC / 19.0) / avg_away_conceded_13
table_13.head()

,Team,HGS,AGS,HAS,AAS,HGC,AGC,HDS,ADS
0,Almeria,26,17,0.838710,0.800000,31,40,1.458824,1.290323
1,Ath Bilbao,42,24,1.354839,1.129412,18,21,0.847059,0.677419
2,Ath Madrid,49,28,1.580645,1.317647,10,16,0.470588,0.516129
3,Barcelona,64,36,2.064516,1.694118,15,18,0.705882,0.580645
4,Betis,19,17,0.612903,0.800000,31,47,1.458824,1.516129


In [12]:
#Expected number of goals based on the average poisson probability
def exp_goals(mean):
    max_pmf = 0;
    for i in xrange(7):
        pmf = scipy.distributions.poisson.pmf(i,mean) * 100 
        if pmf > max_pmf:
            max_pmf = pmf
            goals = i
    return goals

In [13]:
test_13 = res_13.ix[:,0:5]
test_13.head()
test_14 = res_14.ix[:,0:5]
test_14.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Almeria,Espanol,1,1,D
1,Granada,La Coruna,2,1,H
2,Malaga,Ath Bilbao,1,0,H
3,Sevilla,Valencia,1,1,D
4,Barcelona,Elche,3,0,H


In [14]:
table_13[table_13['Team'] == 'Barcelona']
test_14['ER'] = ''


In [15]:
results = []
for index, row in test_14.iterrows():

    home_team = table_13[table_13['Team'] == row['HomeTeam']]
    away_team = table_13[table_13['Team'] == row['AwayTeam']]
    #print "Home : ", home_team.HAS.values, "Away: ", away_team.AAS.
     
    if row.HomeTeam not in ['Leicester', 'QPR', 'Burnley'] and row.AwayTeam not in ['Leicester', 'QPR', 'Burnley']:
        EH = home_team.HAS.values * away_team.ADS.values * avg_home_scored_13
        EA = home_team.HDS.values * away_team.AAS.values * avg_home_conceded_13
        #print row.HomeTeam, row.AwayTeam
        if exp_goals(EH) > exp_goals(EA):
            results.append('H')
        elif exp_goals(EH) < exp_goals(EA):
            results.append('A')
        else:
            results.append('D')
    else:
        results.append('D')

In [16]:
len(results)

380

In [35]:
test_14['ER'] = results

In [36]:
test_14[test_14["HomeTeam"]=='Barcelona']

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,ER
4,Barcelona,Elche,3,0,H,H
21,Barcelona,Ath Bilbao,2,0,H,D
53,Barcelona,Granada,6,0,H,H
72,Barcelona,Eibar,3,0,H,H
92,Barcelona,Celta,0,1,A,D
112,Barcelona,Sevilla,5,1,H,H
134,Barcelona,Espanol,5,1,H,D
151,Barcelona,Cordoba,5,0,H,H
176,Barcelona,Ath Madrid,3,1,H,H
205,Barcelona,Villarreal,3,2,H,D


In [19]:
from sklearn.metrics import accuracy_score


In [95]:
curr_table_14 = pd.DataFrame(columns=('Team','Streak','Position','Form'))

In [96]:
test_14.HomeTeam.unique()

array(['Almeria', 'Granada', 'Malaga', 'Sevilla', 'Barcelona', 'Celta',
       'Eibar', 'Levante', 'Real Madrid', 'Vallecano', 'Getafe',
       'Valencia', 'Ath Bilbao', 'Ath Madrid', 'Cordoba', 'Espanol',
       'Elche', 'La Coruna', 'Sociedad', 'Villarreal'], dtype=object)

In [103]:
curr_table_14.Team = test_14.HomeTeam.unique()
curr_table_14.Streak = 0
curr_table_14.Form = 0
curr_table_14.sort_values(by='Team',inplace=True)
curr_table_14.Position = xrange(1,21)

In [102]:
curr_table_14

,Team,Streak,Position,Form
0,Almeria,0,1,0
12,Ath Bilbao,0,2,0
13,Ath Madrid,0,3,0
4,Barcelona,0,4,0
5,Celta,0,5,0
14,Cordoba,0,6,0
6,Eibar,0,7,0
16,Elche,0,8,0
15,Espanol,0,9,0
10,Getafe,0,10,0


In [42]:
accuracy_score(test_14[(test_14["HomeTeam"]=='Barcelona') ]['ER'], test_14[(test_14["HomeTeam"]=='Barcelona')]['FTR'])

0.57894736842105265